# Implementación gradiente estocástico
### Javier Andres Téllez Ortiz - 201617861

In [1]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


###### Se descarga el dataset y se descomprime

In [2]:
import wget
from zipfile import ZipFile

##Se descarga el archivo del repositorio 
file = wget.download("http://www3.dsi.uminho.pt/pcortez/wine/winequality.zip")

##Se abre el archivo y se descomprime
zpFile = ZipFile(file)
zpFile.extractall()
zpFile.close()

100% [..............................................................................] 96005 / 96005

###### Se abre el dataset y se verifica la estrutura de los datos

In [3]:
import pandas as pd
import numpy as np
import math

##Se abre el archivo con más datos disponibles
dataframe = pd.read_csv("winequality/winequality-white.csv", delimiter = ";")

##Se muestra la composición de los datos
dataframe.dropna()
dataframe.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


###### Se obtienen los nombres de cada característica y el valor de los datos

In [4]:
##Se obtienen los nombres de los parámetros del modelo del dataset
features = dataframe.columns.tolist()

##Se obtienen los valores y se dividen entre datos y objetivos
values = dataframe.values

y = values[:,-1]
X = values[:,0:-1]

###### Se normalizan los datos y se agrega un vector de 1's que representan el intercepto

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

##Se estandarizan los datos provenientes del archivo
normalized_values = scaler.fit(values).transform(values)

##Se dividen entre datos y objetivos
normalized_y = normalized_values[:,-1]
normalized_X = normalized_values[:,0:-1]

##Se agrega un vector de unos a los datos
data_rows = normalized_X.shape[0]
ones_vector =  np.ones([data_rows,1])
X_ones = np.concatenate((ones_vector , normalized_X), 1)

###### Se obtiene el número de condición de cada conjunto de datos. De este resultado, es posible afirmar que si se desea usar desenso de gradiente conviene usar lo datos estandarizados; por lo que se hará el análisis con estos datos.

In [6]:
condition_number_1 = np.linalg.cond(np.matmul(X.T,X))
condition_number_2 = np.linalg.cond(np.matmul(X_ones.T,X_ones))

print("Número de condición datos sin preprocesar: %d" % (condition_number_1))
print("Número de condición datos preprocesados: %d" % (condition_number_2))

Número de condición datos sin preprocesar: 59401148
Número de condición datos preprocesados: 156


###### Se dividen los datos entre conjunto de prueba y conjunto de entrenamiento

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_ones, normalized_y, test_size = 0.4083, random_state = 7861) 

###### Se define la función que retorna el valor del gradiente para los parámetros y datos ingresados 

In [8]:
def calculate_gradient(X, y, w):
    
    y_hat = np.matmul(X,w)
    error = y_hat - y  
    gradiente = np.matmul(error, X)
    
    return gradiente

###### Se define la función que divide los datos en lotes de acuerdo con el tamaño especificado

In [17]:
def splitBatches(X, y, batch_size):
    ## Se obtiene el número total de datos
    m = X.shape[0]
    
    ##Se calcula la cantidad total de lotes
    batches = math.ceil(m/batch_size)
    
    ##Se mezclan aleatoriamente los datos
    np.random.seed(None)
    data = np.c_[X,y]
    np.random.shuffle(data)

    X_batches = []
    y_batches = []
    
    for i in range(batches):
        
        ##Se toman subconjuntos de datos con el tamaño especificado 
        X_batch = data[0:batch_size,:-1]
        X_batches.append(X_batch)
                
        y_batch = data[0:batch_size,-1]
        y_batches.append(y_batch)
        
        data = data[batch_size:,:]
        
    return X_batches, y_batches


###### Se define la función para realizar el entrenamiento sobre el conjunto de datos

In [18]:
def model_train(X, y, batch_size = 1, epocs = 5, learning_rate = 0.0001, tol = 1e-4):
    
    ##Se define el número de parámetros a entrenar
    n = X.shape[1]
    
    ##Se inicializan los parámetros a entrenar
    np.random.seed(7861)
    w_0 = np.random.rand(n)
    
    ##Se inicializa el error en un valor arbitrario
    actual_error = 2    
    
    for epoc in range(epocs):
        
        ##Se dividen los datos en lotes al inicio de cada época
        ##se realiza este procedimiento antes de cada época para 
        ##garantizar que los datos estén barajados de forma distinta
        ##en cada recorrido
        X_batches, y_batches = splitBatches(X, y, batch_size)
        
        for X_batch, y_batch in zip(X_batches, y_batches):
            
            ##Para cada lote, se realiza el cálculo del gradiente y
            ##la actualización de los pesos
            w = calculate_gradient(X_batch, y_batch, w_0)                  
            w_0 = w_0 - learning_rate * w
        
        ##Se calcula el error al final de cada recorrido por los datos
        previous_error = actual_error
        actual_error = quadratic_error(X, y, w_0)
        
        ##Se imprime el valor del error cada 100 épocas
        if (epoc % 100 == 0):     
            print("Epoc %d error: %.4f" % (epoc,actual_error))
            
        ##Se verifica que el cambio del error no sea menor a la toleracia
        ##cada 20 épocas
        if (epoc % 20 == 0 and np.abs(previous_error - actual_error) < tol):
            break
         
    ##Se reporta el error del modelo completamente entrenado
    print("Model error : %.4f" % (actual_error))
    return w_0

###### Se define una fución que permite calcular el error cuadrático 

In [19]:
def quadratic_error(X,y,w):
    y_hat = np.matmul(X,w)
    error = y_hat - y
    error = np.mean(error ** 2)
    return error

###### Se entrena el modelo con un tamaño de lote de 10, una tasa de aprendizaje de  10^-4 (máxima tasa encontrada que garantiza convergencia), 1000 recorridos por todos los datos de entrenamiento y un criterio de parada de 10^-6 para el error 

In [20]:
W = model_train(X_train, y_train, batch_size = 10, learning_rate=0.0001, epocs = 1000, tol = 1e-6)

Epoc 0 error: 3.0090
Epoc 100 error: 0.6975
Epoc 200 error: 0.6948
Epoc 300 error: 0.6942
Epoc 400 error: 0.6940
Model error : 0.6940


###### Se entrena otro modelo haciendo uso de la librería skleran usando el mismo conjunto de datos

In [21]:
from sklearn.linear_model import LinearRegression
regression = LinearRegression().fit(X_train[:,1:], y_train)

###### Se reportan los coeficientes obtenidos los cuales poseen valores muy cercanos 

In [14]:
regression_model = pd.DataFrame({"Característica" : features[0:-1],
                                "Gradiente estocástico" : W[1:],
                                "Librería" : regression.coef_ 
                                })

regression_model

,Característica,Gradiente estocástico,Librería
0,fixed acidity,0.013082,0.033863
1,volatile acidity,-0.228133,-0.225578
2,citric acid,0.002000,0.003471
3,residual sugar,0.359827,0.420781
4,chlorides,-0.017378,-0.014071
5,free sulfur dioxide,0.093169,0.088576
6,total sulfur dioxide,-0.025303,-0.020836
7,density,-0.320897,-0.410741
8,pH,0.080547,0.098020
9,sulphates,0.060552,0.066715


###### Se calcula el error cuadrático medio para ambos modelos y el valor de R2

In [23]:
from sklearn.metrics import r2_score

y_hat = regression.predict(X_test[:,1:])
sk_error = y_hat - y_test
sk_error = np.mean(sk_error ** 2)

estocastic_error = quadratic_error(X_test, y_test, W)
y_hat_est = np.matmul(X_test,W)

r2_sk = r2_score(y_test, y_hat)
r2_est = r2_score(y_test, y_hat_est)

models_error = pd.DataFrame({"Tipo de entrenamiento" : ["Gradiente estocástico", "Librería"],
                            "Error cuadrático" :[estocastic_error, sk_error],
                            "R2" : [r2_est, r2_sk]})

models_error

,Tipo de entrenamiento,Error cuadrático,R2
0,Gradiente estocástico,0.756367,0.251742
1,Librería,0.755971,0.252133


## Conclusiones:
- Se evidencia que la implementación realizada del algoritmo de descenso de gradiente estocástico es correcta debido a que genera resultados similares a los de la librería sklearn.
- Se observó la importancia de la escogencia adecuada de parámetros como la tasa de aprendizaje, el tamaño de los lotes y el número de épocas de entrenamiento; ya que estas afectan la velocidad de convergencia del algortimo y la precisión de la aproximación al modelo real del fenómeno que describen los datos.
- Nuevamente es posible afirmar, a partir del error obtenido y del R2, que el modelo lineal no es un modelo adecuado para el modelamiento del fenómeno estudiado.